In [ ]:
import numpy as np

class Evaluation_Metric:
    
    def __init__(self,account):
        self.account = account
        
    
#   年化收益率=[（投资收益 / 本金）/ 投资天数] × 一年天数 ×100%
    def year_return_rate(self):
        asset_df = self.account.get_asset_df()
        
        last_day_asset = asset_df[asset_df.shape[0]-1:]        
        invest_return = last_day_asset['assets'].values[0]
        
        total_days = asset_df.shape[0]
        
        invest_rate = (invest_return-self.account.init_money)*1.0/self.account.init_money
        year_return_rate_value = 100.0 * 365 * (invest_rate/total_days)
        
        return round(year_return_rate_value,2)
    
    
#   月回报率
    def month_return_rate_array(self):
        asset_df = self.account.asset
        
        asset_df['year'] = asset_df['date'].apply(lambda x: int(x.split('-')[0]))
        asset_df['month'] = asset_df['date'].apply(lambda x: int(x.split('-')[1]))

        years = set(asset_df['year'].values)
        years = list(years)
        years.sort()

        rate_array = []
        
        for each_year in years:
            year_part = asset_df[asset_df['year']==each_year]
            
            months = set(year_part['month'].values)
            months = list(months)
            months.sort()
            
            month_array= []
            for each_month in months:
                month_part = year_part[year_part['month']==each_month]
                
                first_day_of_month = month_part[:1]['assets'].values[0]
                last_day_of_month = month_part[month_part.shape[0]-1:]['assets'].values[0]
                
                month_return_rate = round(100*(last_day_of_month-first_day_of_month)/first_day_of_month,2)

                month_array.append(month_return_rate)
                
            rate_array += month_array
        
        return rate_array
        
    
#   年回报率
    def year_return_rate_array(self):
        asset_df = self.account.asset
        
        asset_df['year'] = asset_df['date'].apply(lambda x: int(x.split('-')[0]))

        years = set(asset_df['year'].values)
        years = list(years)
        years.sort()

        rate_array = []
        
        for each_year in years:
            year_part = asset_df[asset_df['year']==each_year]

            first_day_of_year = year_part[:1]['assets'].values[0]
            last_day_of_year = year_part[year_part.shape[0]-1:]['assets'].values[0]

            year_return_rate = round(100*(last_day_of_year-first_day_of_year)/last_day_of_year,2)
            
            rate_array.append(year_return_rate)
        
        return rate_array
    
    def mean_year_return_rate(self):
        year_returns = self.year_return_rate_array()
        return np.mean(year_returns)
    
    def std_year_return_rate(self):
        year_returns = self.year_return_rate_array()
        return np.std(year_returns,ddof=1)
    
    
    def mean_month_return_rate(self):
        month_returns = self.month_return_rate_array()
        return np.mean(month_returns)
    
    def std_month_return_rate(self):
        month_returns = self.month_return_rate_array()
        return np.std(month_returns,ddof=1)
    
        
    def continue_loss_times(self,profits):
        max_loss_count = 0
        
        for i in range(1,len(profits)):
            cur_profit = profits[i]
            
            if(cur_profit<=0):
                loss_count = 0
                j=i;
                while(j>=0 and profits[j]<0):
                    loss_count+=1
                    j -= 1
                if(loss_count>max_loss_count):
                    max_loss_count = loss_count
                
        return max_loss_count
    
    def max_drawdown_per(self, array):
        max_asset = np.maximum.accumulate(array)
        
        i = np.argmax((max_asset - array)/max_asset) # end of the period
        j = np.argmax(array[:i]) # start of period
        
        return round((1-array[i]/array[j])*100.0,2)

    
    def max_drawdown(self):
        asset_df = self.account.get_asset_df()
        assets_array = asset_df['assets'].values
        
        return self.max_drawdown_per(assets_array)
    
        